In [74]:
import requests
from time import sleep
import pandas as pd
from datetime import datetime
import os 


In [75]:
api_key = "RGAPI-69560027-3975-44a0-a1df-717f5ad91963"

In [76]:
servers ={
    "br1":"br1",
     "eun1":"eun1",
     "euw1":"euw1",
     "la1":"la1",
     "la2":"la2",
     "na1":"na1",
     "oc1":"oc1",
     "ru":"ru",
     "tr1":"tr1",
}

In [77]:
name = "alemsd14"
user_per_server = {}
for server in servers:
    endpoint=f"https://{server}.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}?api_key=RGAPI-69560027-3975-44a0-a1df-717f5ad91963"
    res = requests.get(endpoint).json()
    if res.get("status", None):
        print(f"No user in {server}:{res}")
    else:
        print(f"User found in {server}")
        user_per_server[server] = res
    sleep(1)


No user in br1:{'status': {'message': 'Data not found - summoner not found', 'status_code': 404}}
No user in eun1:{'status': {'message': 'Data not found - summoner not found', 'status_code': 404}}
No user in euw1:{'status': {'message': 'Data not found - summoner not found', 'status_code': 404}}
No user in la1:{'status': {'message': 'Data not found - summoner not found', 'status_code': 404}}
User found in la2
No user in na1:{'status': {'message': 'Data not found - summoner not found', 'status_code': 404}}
No user in oc1:{'status': {'message': 'Data not found - summoner not found', 'status_code': 404}}
No user in ru:{'status': {'message': 'Data not found - summoner not found', 'status_code': 404}}
No user in tr1:{'status': {'message': 'Data not found - summoner not found', 'status_code': 404}}


In [78]:
user_per_server

{'la2': {'id': 'qjeK5zC-LurB2kx56qh3aJvsQjvkmHoFEQKrP5idLclf7xY',
  'accountId': 'q3s3UKNOkRDm02UkGMJhXnsWInVAwk5xutffcBaTxZsaV_Q',
  'puuid': '8d3WIsnmge6BGLXHsRNbV1hW6itmW_60bDRGARpFTxZBvdjbDL2PMG9MuuWxnl-l7X5OPAqbhwGxxQ',
  'name': 'alemsd14',
  'profileIconId': 5269,
  'revisionDate': 1675920123000,
  'summonerLevel': 125}}

##Account

In [79]:
endpoint=f"https://la2.api.riotgames.com/lol/summoner/v4/summoners/by-name/alemsd14?api_key=RGAPI-69560027-3975-44a0-a1df-717f5ad91963"
summoner = requests.get(endpoint).json()

In [80]:
df=pd.DataFrame([summoner])
df

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel
0,qjeK5zC-LurB2kx56qh3aJvsQjvkmHoFEQKrP5idLclf7xY,q3s3UKNOkRDm02UkGMJhXnsWInVAwk5xutffcBaTxZsaV_Q,8d3WIsnmge6BGLXHsRNbV1hW6itmW_60bDRGARpFTxZBvd...,alemsd14,5269,1675920123000,125


In [81]:
summ_id = "qjeK5zC-LurB2kx56qh3aJvsQjvkmHoFEQKrP5idLclf7xY" 

##Extract and concat ranking mastery

In [82]:
summ_id=summoner["id"]
endpoint=f"https://la2.api.riotgames.com/lol/league/v4/entries/by-summoner/{summ_id}?api_key=RGAPI-69560027-3975-44a0-a1df-717f5ad91963"
res = requests.get(endpoint).json()
df_rank = pd.DataFrame([res[0]])
df_rank

,leagueId,queueType,tier,rank,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,81823043-4d28-4c75-aa34-64a0164fb54c,RANKED_SOLO_5x5,BRONZE,III,qjeK5zC-LurB2kx56qh3aJvsQjvkmHoFEQKrP5idLclf7xY,alemsd14,26,18,21,False,False,False,False


In [83]:
df_summ = pd.concat([df,df_rank],axis=1)

In [84]:
df_summ.dtypes

id               object
accountId        object
puuid            object
name             object
profileIconId     int64
revisionDate      int64
summonerLevel     int64
leagueId         object
queueType        object
tier             object
rank             object
summonerId       object
summonerName     object
leaguePoints      int64
wins              int64
losses            int64
veteran            bool
inactive           bool
freshBlood         bool
hotStreak          bool
dtype: object

In [85]:
df_summ=df_summ.drop(columns=["summonerId","summonerName"])
df_summ

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel,leagueId,queueType,tier,rank,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,qjeK5zC-LurB2kx56qh3aJvsQjvkmHoFEQKrP5idLclf7xY,q3s3UKNOkRDm02UkGMJhXnsWInVAwk5xutffcBaTxZsaV_Q,8d3WIsnmge6BGLXHsRNbV1hW6itmW_60bDRGARpFTxZBvd...,alemsd14,5269,1675920123000,125,81823043-4d28-4c75-aa34-64a0164fb54c,RANKED_SOLO_5x5,BRONZE,III,26,18,21,False,False,False,False


In [86]:
#lets transform revisionDate from epoch to datetime
df["revisionDate"] = pd.to_datetime(df["revisionDate"],unit="ms")
df["revisionDate"]

0   2023-02-09 05:22:03
Name: revisionDate, dtype: datetime64[ns]

Save the data into a file

In [87]:

df.to_csv(f"data/summoner.csv",index = False)

##Champions mastery

In [88]:

endpoint=f"https://la2.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-summoner/{summ_id}?api_key=RGAPI-69560027-3975-44a0-a1df-717f5ad91963"
res = requests.get(endpoint).json()
df_champs =pd.DataFrame(res)
df_champs

,championId,championLevel,championPoints,lastPlayTime,championPointsSinceLastLevel,championPointsUntilNextLevel,chestGranted,tokensEarned,summonerId
0,131,7,170744,1675413789000,149144,0,True,0,qjeK5zC-LurB2kx56qh3aJvsQjvkmHoFEQKrP5idLclf7xY
1,63,7,126886,1675483843000,105286,0,True,0,qjeK5zC-LurB2kx56qh3aJvsQjvkmHoFEQKrP5idLclf7xY
2,99,6,72363,1675898121000,50763,0,False,0,qjeK5zC-LurB2kx56qh3aJvsQjvkmHoFEQKrP5idLclf7xY
3,22,5,63141,1659714423000,41541,0,False,0,qjeK5zC-LurB2kx56qh3aJvsQjvkmHoFEQKrP5idLclf7xY
4,25,6,45666,1660340131000,24066,0,False,2,qjeK5zC-LurB2kx56qh3aJvsQjvkmHoFEQKrP5idLclf7xY
...,...,...,...,...,...,...,...,...,...
57,42,1,109,1594768865000,109,1691,False,0,qjeK5zC-LurB2kx56qh3aJvsQjvkmHoFEQKrP5idLclf7xY
58,80,1,100,1593908980000,100,1700,False,0,qjeK5zC-LurB2kx56qh3aJvsQjvkmHoFEQKrP5idLclf7xY
59,122,1,95,1614615851000,95,1705,False,0,qjeK5zC-LurB2kx56qh3aJvsQjvkmHoFEQKrP5idLclf7xY
60,92,1,89,1594769967000,89,1711,False,0,qjeK5zC-LurB2kx56qh3aJvsQjvkmHoFEQKrP5idLclf7xY


In [89]:
df_champs["lastPlayTime"] = pd.to_datetime(df_champs["lastPlayTime"],unit="ms")
df.to_csv(f"data/champs_mastery_{datetime.today()}.csv",index = False)


##Match history

In [98]:
puuid=summoner["puuid"]
endpoint =f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/8d3WIsnmge6BGLXHsRNbV1hW6itmW_60bDRGARpFTxZBvdjbDL2PMG9MuuWxnl-l7X5OPAqbhwGxxQ../ids?start=0&count=20&api_key=RGAPI-69560027-3975-44a0-a1df-717f5ad91963"
res = requests.get(endpoint).json()


In [99]:
res

['LA2_1267460200',
 'LA2_1267324557',
 'LA2_1267307174',
 'LA2_1265790642',
 'LA2_1265779190',
 'LA2_1265734490',
 'LA2_1265713583',
 'LA2_1265537193',
 'LA2_1265535067',
 'LA2_1265533218',
 'LA2_1265527855',
 'LA2_1265141971',
 'LA2_1265125268',
 'LA2_1264943553',
 'LA2_1264252660',
 'LA2_1264240927',
 'LA2_1264232571',
 'LA2_1264219733',
 'LA2_1264208461',
 'LA2_1264199296']

In [102]:
match_id=res[0]
endpoint =f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key=RGAPI-69560027-3975-44a0-a1df-717f5ad91963"
res = requests.get(endpoint).json()
res

{'metadata': {'dataVersion': '2',
  'matchId': 'LA2_1267460200',
  'participants': ['24i-lojk8aPHF7Iv90sHezj5ellFFVWH7LbcwuRgXwITSn7OGaw2SrRpkDUek6odMKRUHJ-ZlmSShQ',
   'xG5_J99inNfi5IIV-wZSLS74PEWg5IdMgazi26xUdgcBAyQRRnhMu5iDjm1KtPVSutYzp1CHHJt4jQ',
   'CVv9BdTxSbUkF_A69clWVGhEf194ZXG9BdaC2PWeX0VJuUkcG04UT9OrPkIxnIa3usyg4-RkAETSAQ',
   'fG3DoLQu8ZCEHG01A1hSegW2zeffnhmso983wlhzNDENX94opu8IpBVYfgVrMWPWb4zskrqR5gNVDg',
   'drqjXE-TGMxSiytjfJ089kGE-tSeQx4i45rZ8gfDeP03GVUR8tpSXwbEYVZLtuiBM2vqoaWGbjVfXg',
   'fl1GN1oJrsXsl06u0M9TqhAVfiSoz4mOcum8bAukTzsL_91-d6CjqKdgwwnOaJGWTzl-GR1T8NYuRg',
   'ZouJZk-Gp8IXcMO2O8pWFABspTusUw0Ic8iyqIlKHe2JAM6tcp3bC6JUtIT41v1SNvHoYlixyNZCJg',
   '8d3WIsnmge6BGLXHsRNbV1hW6itmW_60bDRGARpFTxZBvdjbDL2PMG9MuuWxnl-l7X5OPAqbhwGxxQ',
   '4dlNzWcRrJBJENaSMK5wTvli2J5P5ghHIGurh4yWHuC_PWTyvxLbbJbKkAslNZHNV0J4_aKucNM_sw',
   'VhItSDp53WImNiVgTy4pJy4BHorkevgm3HC8iM-0WW3H1AZ45l5JKTo1yUYgY-gHmGdNLM1MkthyWA']},
 'info': {'gameCreation': 1675917765843,
  'gameDuration': 2210,
  

In [119]:
basics = [
    'puuid',
     "win",
     "lane",
      "role",
      "teamPosition",
      "kills",
      "deaths",
      "totalMinionsKilled",
      'eligibleForProgression',
       "timePlayed",
      'gameEndedInEarlySurrender',
         'gameEndedInSurrender',


  ]

kills =[
      "killingSprees",
      "firstBloodAssist",
      "firstBloodKill",
      "doubleKills",
      "tripleKills",
      "quadraKills",
       "pentaKills",
  ]

wards=[
      "wardsKilled",
      "wardsPlaced",
      "visionWardsBoughtInGame",
      "detectorWardsPlaced",
  ]

objetives = [
      "objectivesStolen",
      "turretKills",
      "dragonKills",
      "inhibitorKills",
      "nexusKills",
  ]

pings =[
      "dangerPings",
      "enemyMissingPing",
      "enemyVisionPings",
      "getBackPings",
      "holdPings",
      "needVisionPings",
      "onMyWayPings",
      "pushPings",
      "visionClearedPings",
  ]

stats = basics + kills +wards + objetives

In [120]:
df = pd.DataFrame(res["info"]["participants"])
df = df[stats]
df

,puuid,win,lane,role,teamPosition,kills,deaths,totalMinionsKilled,eligibleForProgression,timePlayed,...,pentaKills,wardsKilled,wardsPlaced,visionWardsBoughtInGame,detectorWardsPlaced,objectivesStolen,turretKills,dragonKills,inhibitorKills,nexusKills
0,24i-lojk8aPHF7Iv90sHezj5ellFFVWH7LbcwuRgXwITSn...,True,MIDDLE,DUO,TOP,14,8,175,True,2210,...,0,0,14,5,5,0,2,0,1,0
1,xG5_J99inNfi5IIV-wZSLS74PEWg5IdMgazi26xUdgcBAy...,True,JUNGLE,NONE,JUNGLE,8,10,25,True,2210,...,0,2,12,0,0,0,1,2,0,0
2,CVv9BdTxSbUkF_A69clWVGhEf194ZXG9BdaC2PWeX0VJuU...,True,MIDDLE,DUO,MIDDLE,7,9,139,True,2210,...,0,0,8,0,0,0,2,0,2,0
3,fG3DoLQu8ZCEHG01A1hSegW2zeffnhmso983wlhzNDENX9...,True,BOTTOM,CARRY,BOTTOM,13,8,196,True,2210,...,0,2,10,0,0,1,5,1,3,0
4,drqjXE-TGMxSiytjfJ089kGE-tSeQx4i45rZ8gfDeP03GV...,True,BOTTOM,SUPPORT,UTILITY,1,7,35,True,2210,...,0,7,21,0,0,0,0,0,0,1
5,fl1GN1oJrsXsl06u0M9TqhAVfiSoz4mOcum8bAukTzsL_9...,False,TOP,SOLO,TOP,4,7,112,True,2210,...,0,0,12,0,0,0,1,0,0,0
6,ZouJZk-Gp8IXcMO2O8pWFABspTusUw0Ic8iyqIlKHe2JAM...,False,JUNGLE,NONE,JUNGLE,19,8,55,True,2210,...,0,1,8,0,0,0,0,3,0,0
7,8d3WIsnmge6BGLXHsRNbV1hW6itmW_60bDRGARpFTxZBvd...,False,MIDDLE,SOLO,MIDDLE,10,12,132,True,2210,...,0,0,15,4,4,0,1,0,0,0
8,4dlNzWcRrJBJENaSMK5wTvli2J5P5ghHIGurh4yWHuC_PW...,False,BOTTOM,CARRY,BOTTOM,6,7,202,True,2210,...,0,1,13,2,2,0,2,0,0,0
9,VhItSDp53WImNiVgTy4pJy4BHorkevgm3HC8iM-0WW3H1A...,False,BOTTOM,SUPPORT,UTILITY,3,10,63,True,2210,...,0,1,24,0,0,0,0,0,0,0
